In [12]:
#All libs used to analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from sqlalchemy import create_engine, text
import pandas as pd

In [13]:
#Credentials
postgres_dbname = 'clever'
postgres_user = 'clever'
postgres_password = 'clever'
postgres_host = 'postgres_clever'
postgres_port = '5432'

#All avaiable tables into Postgress SQL for checking
avaiable_tables = [
    'fmcsa_complaints',
    'fmcsa_safer_data',
    'fmcsa_company_snapshot',
    'fmcsa_companies',
    'customer_reviews_google',
    'company_profiles_google_maps'
]

In [14]:
#SQL engine, connection and collecting dataframes for analysis
engine = create_engine(
    f'postgresql+psycopg2://{postgres_user}:{postgres_password}@{postgres_host}:{postgres_port}/{postgres_dbname}'
)

with engine.connect() as conn:
    complaints_df = pd.read_sql('SELECT * FROM fmcsa_complaints', conn)
    safer_data_df = pd.read_sql('SELECT * FROM fmcsa_safer_data', conn)
    companies_df = pd.read_sql('SELECT * FROM fmcsa_companies', conn)
    company_snap = pd.read_sql('SELECT * FROM fmcsa_company_snapshot', conn)
    company_df = pd.read_sql('SELECT * FROM company_profiles_google_maps', conn)
    reviews_df = pd.read_sql('SELECT * FROM customer_reviews_google', conn)
    df_join = pd.read_sql("""SELECT cn.*, s.entity_type, s.legal_name, s.operating_status FROM fmcsa_companies cn
                                     LEFT JOIN fmcsa_safer_data s on cn.usdot_num = s.usdot_num 
                                    """, conn)

In [16]:
def rank_complaints(df, column, city=None, state=None):
    
    df[column] = pd.to_numeric(df[column], errors='coerce')
    df = df.dropna(subset=[column])

    # Filter by city or state if provided
    if city:
        df = df[df['city'] == city]
    if state:
        df = df[df['state'] == state]
        
    # Sort by column
    df = df.sort_values(by=[column], ascending=False)

    return df

group_df = df_join.groupby(['entity_type','company_name']).max().reset_index()
ranked_total_complaints = rank_complaints(group_df,'total_complaints_2021')
ranked_total_complaints

,entity_type,company_name,usdot_num,user_created,date_created,user_updated,date_updated,company_url,city,state,total_complaints_2021,total_complaints_2022,total_complaints_2023,location,company_type,legal_name,operating_status
80,BROKER,US STANDARD MOVING & STORAGE CORP,3307068,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.655Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.942Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,MIAMI,FL,70,2,0,"MIAMI, FL",0,US STANDARD MOVING & STORAGE CORP,NOT AUTHORIZED
50,BROKER,MOVING APT INC,2247863,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.523Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.616Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,MIAMI,FL,30,12,4,"MIAMI, FL",0,MOVING APT INC,"AUTHORIZED FOR BROKER Property, HHG"
52,BROKER,MOVING SERVICES,2886867,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.533Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.627Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,MIAMI,FL,14,4,3,"MIAMI, FL",0,ADAMS VAN LINES CORP,AUTHORIZED FOR BROKER HHG
92,CARRIER,A1A MOVERS LLC,2821065,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.137Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.226Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,MIAMI,FL,12,12,0,"MIAMI, FL",2,A1A MOVERS LLC,AUTHORIZED FOR HHG
171,CARRIER,EASY ROAD MOVING & STORAGE INC,3491781,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.311Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.400Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,MIAMI,FL,12,11,7,"MIAMI, FL",8,EASY ROAD MOVING & STORAGE INC,AUTHORIZED FOR HHG
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,CARRIER,C&I EXPRESS TRUCKING,1618312,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:14.890Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:11.162Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,DALLAS,TX,0,0,0,"DALLAS, TX",2,WILLIE NICHOLS,NOT AUTHORIZED
134,CARRIER,BRITESTAR TRANSPORT LLC,4006025,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:49.216Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:44.299Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,MIAMI,FL,0,0,0,"MIAMI, FL",1,BRITESTAR TRANSPORT LLC,NOT AUTHORIZED
133,CARRIER,BP AUCTIONS,3715805,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:14.882Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:11.154Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,DALLAS,TX,0,0,0,"DALLAS, TX",2,BP AUCTIONS LLC,NOT AUTHORIZED
132,CARRIER,BOLT MOVERS,3545789,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:39:14.879Z,88f52f6b-b3f1-4ef4-ad24-a605f568e3ef,2024-02-16T17:52:11.151Z,https://ai.fmcsa.dot.gov/hhg/SearchDetails.asp...,DALLAS,TX,0,0,0,"DALLAS, TX",2,BOLT MOVERS DALLAS LLC,AUTHORIZED FOR HHG


In [17]:
#Ranked reviews by rating descending
def rank_reviews(df, city=None, state=None):
    
    df['rating'] = pd.to_numeric(df['rating'], errors='coerce')
    df = df.dropna(subset=['rating'])
    
    # Ensure rating is between 1 >= integer <= 5
    df = df[(df['rating'].astype(int) >= 1) & (df['rating'].astype(int) <= 5)]
    
    # Filter by city or state if provided
    if city:
        df = df[df['city'] == city]
    if state:
        df = df[df['state'] == state]
        
    # Sort by rating to see the best rating companies
    df = df.sort_values(by=['rating'], ascending=False)

    return df[['name', 'reviews_id', 'rating', 'city', 'state']]


ranked_reviews = rank_reviews(company_df)
ranked_reviews['city'].unique()
# Here you can see the citys and choose one for filtering 

array(['Miami', 'Dallas', 'Seattle'], dtype=object)

In [18]:
ranked_reviews['state'].unique()
# Here you can see the states and choose one for filtering 

array(['Florida', 'Texas', 'Georgia', 'Indiana', 'Oklahoma', 'FL', 'TX',
       'Washington', 'Oregon', 'Pennsylvania', 'WA', 'North Carolina'],
      dtype=object)

In [19]:
rank_reviews(company_df)

,name,reviews_id,rating,city,state
0,Nicolas Boucher P.A,None,5.0,Miami,Florida
1957,Monument Realty - Dallas,None,5.0,Dallas,Texas
1984,"Matt Twomey, Highlands Real Estate",None,5.0,Dallas,Texas
1981,Cecilia Labossiere-Dallas Ft. Worth-Realtor,None,5.0,Dallas,Texas
1980,The Realm Agency,None,5.0,Dallas,Texas
...,...,...,...,...,...
725,Alexandria Real Estate Equities,8.82389343379695E18,1.0,Seattle,Washington
3745,Sun Realty Investments Inc,1.36096E19,1.0,Miami,Florida
1163,Dalrock Properties LLC,None,1.0,Dallas,Texas
1159,First Industrial Realty Trust,1.2609184836918764E19,1.0,Dallas,Texas


In [20]:
rank_reviews(company_df, city = 'Miami')

,name,reviews_id,rating,city,state
0,Nicolas Boucher P.A,None,5.0,Miami,Florida
3444,Mireille P. Segovia Realtor,8.57802E18,5.0,Miami,Florida
3410,Karina Castillo Realtor - PA,None,5.0,Miami,FL
3411,Miami es tu Inversión,None,5.0,Miami,Florida
3413,Lidia Ivonne Garza Realtor,None,5.0,Miami,Florida
...,...,...,...,...,...
3958,"Epic Realty Services, Inc.",6.27208E18,1.0,Miami,Florida
2884,"Core Investment Management, LLC",None,1.0,Miami,Florida
2361,Keller Williams Realty: Ronald Platt,1.68567E19,1.0,Miami,Florida
2274,Cynthia Caridad P.A.,None,1.0,Miami,Florida


In [21]:
# Sentiment Analysis

# Getting only matching ids, because reviews have a lot of issues in google_id column
reviews_df = reviews_df.loc[reviews_df['google_id'].str.startswith('0')]
company_df = company_df.loc[company_df['google_id'].str.startswith('0')]

# Getting the review_text for sentiment analysis
df2 = pd.merge(company_df, reviews_df[['google_id','review_text']],on='google_id', how='left')
df2 = df2.loc[~df2['review_text'].isnull()]

def analyze_sentiment(reviews):
    # Initialize VADER sentiment intensity analyzer
    analyzer = SentimentIntensityAnalyzer()
    
    # Apply sentiment analysis to each review in the 'review_text' column
    reviews['sentiment'] = reviews['review_text'].apply(lambda x: analyzer.polarity_scores(x)['compound'])
    
    # Categorize sentiment based on the compound score
    reviews['sentiment_category'] = reviews['sentiment'].apply(
        lambda score: 'positive' if score > 0.05 else ('negative' if score < -0.05 else 'neutral')
    )
    
    return reviews
 
df = analyze_sentiment(df2)
reviews_analysis = df[['google_id','name','rating','review_text','sentiment_category']]
pd.set_option('display.max_colwidth', None)
reviews_analysis.head(5)


,google_id,name,rating,review_text,sentiment_category
43,0x54900fa7f798bdd1:0x778e5ec2841c59e8,Pickett Street Properties with Keller Williams Greater Seattle,4.7,Pickett Street Properties were very accommodating considering I was out of the country for all the documents to be signed remotely. Also there was a tenant that was procrastinating his move which required a lot of patience. Jesse helped take care of all upgrades and other necessities to make the sale go through seamlessly. And I got a better sale than the original listing!,positive
44,0x54900fa7f798bdd1:0x778e5ec2841c59e8,Pickett Street Properties with Keller Williams Greater Seattle,4.7,"Jillian Farrar was recommended to us by a family member, and it turned out to be the best partnership we could have imagined. We had sold our house in the SF Bay Area with the intent of relocating back to the NW after 45 years of CA living. To say things had changed in those years would have been an understatement, when did Newcastle become a city, or all of these unincorporated areas garner names?! We were not sure when we came up here if we should buy at basically the top of the market, or rent for a period of time to see if the market cooled and prices came down. What this meant was even as we looked around at potential areas to live in, primarily the Eastside but also Pierce County (for housing affordability) we would think about places to rent as well. Jillian had the flexibility and patience to deal with our almost daily change of plans. Through it all she kept us focused on what we had provided as key “must haves” . We went to several houses Jillian gave us leads on, and she coordinated visits to houses we found and thought were good fits. The house we are in now was a place that she coordinated our viewing of as we were coming back from a visit to another house about an hour away. Thanks to Jillian, our new home met our needs, and has so many more attributes. We are in a house that we can age in (no stairs, single floor), upgraded kitchens and bathrooms, move-in condition, a large flat backyard with minimal seasonal maintenance requirements but most importantly within 10 minutes of our first grandchild, the reason we were moving in the first place. Thank you Jillian and the staff at Pickett Street Properties for; as we used to say in business, meeting our specifications and exceeding our expectations.",positive
45,0x54900fa7f798bdd1:0x778e5ec2841c59e8,Pickett Street Properties with Keller Williams Greater Seattle,4.7,My husband and I worked with Jillian Farrar to purchase a home in Spring 2022. She was great to work with and made a potentially stressful process much more smooth and enjoyable. She was willing to go the extra mile and work quickly to secure our home in a competitive market. She has extensive experience and is knowledgeable about the intricacies of real estate transactions. I would 110% work with Jillian and the Pickett Street team again and would recommend them to anyone looking for a home in the Puget Sound region.,positive
46,0x54900fa7f798bdd1:0x778e5ec2841c59e8,Pickett Street Properties with Keller Williams Greater Seattle,4.7,"My husband and I interviewed four agents before choosing to go with Jesse Moore at Picket Street Properties. We liked his approach, being straight forward and we could tell he was very knowledgeable in the business.\r\nJesse and team were very well prepared and always available to take calls/texts and answer questions quickly.\r\nWe moved out of state before our house was on the market as Jesse put our minds at ease that he and team would take care of everything and they did!\r\nNo worry about cleaning carpets, etc while moving out as they had the house professionally cleaned, took care of staging set up (which was amazing), professional pictures along with two open houses. Our house was under contract within two weeks or so and this was at a time when the market was very unpredictable. We couldn’t have asked for a better team, thank